In [1]:
import pandas as pd
import numpy as np
import sys
%matplotlib inline

In [2]:
print('Python version ' + sys.version)
print('Pandas version ' + pd.__version__)

Python version 3.6.1 | packaged by conda-forge | (default, May 23 2017, 14:31:56) 
[GCC 4.2.1 Compatible Apple LLVM 6.1.0 (clang-602.0.53)]
Pandas version 0.20.2


# 选择

### 如何从一个组中随机的选择一些样本？

In [3]:
# 初始化一个 dataframe
df = pd.DataFrame({'group1' : ["a","b","a","a","b","c","c","c","c",
                            "c","a","a","a","b","b","b","b"],
                'group2' : [1,2,3,4,1,3,5,6,5,4,1,2,3,4,3,2,1],
                'value'  : ["apple","pear","orange","apple",
                            "banana","durian","lemon","lime",
                            "raspberry","durian","peach","nectarine",
                            "banana","lemon","guava","blackberry","grape"]})
df

,group1,group2,value
0,a,1,apple
1,b,2,pear
2,a,3,orange
3,a,4,apple
4,b,1,banana
5,c,3,durian
6,c,5,lemon
7,c,6,lime
8,c,5,raspberry
9,c,4,durian


In [4]:
# 我们不想只是简单地从 df 中随机选择一些行
# 我们想把 df 先按照 (group & group2) 进行分组然后在随机选择一些行
from random import choice

# 先分组
grouped = df.groupby(['group1','group2'])
grouped.size()

#注意，group(a, 1)有两个可能性
#注意，group(a, 2)有一个可能性

#这意味着，如果我们从 group(a, 1) 中随机选择，可能是“apple”或者“peach”
#如果我们从 group(a, 2) 中随机选择，总会是“nectarine”

group1  group2
a       1         2
        2         1
        3         2
        4         1
b       1         2
        2         2
        3         1
        4         1
c       3         1
        4         1
        5         2
        6         1
dtype: int64

In [5]:
#df.loc[从分组中随机选择记录]
df.loc[[choice(x) for x in grouped.groups.values()]]

,group1,group2,value
10,a,1,peach
11,a,2,nectarine
2,a,3,orange
3,a,4,apple
4,b,1,banana
1,b,2,pear
14,b,3,guava
13,b,4,lemon
5,c,3,durian
9,c,4,durian


### 如何对一列中的每一行做切片?

In [6]:
df = pd.DataFrame(data=['abcdef']*10, columns=['text'])
df

,text
0,abcdef
1,abcdef
2,abcdef
3,abcdef
4,abcdef
5,abcdef
6,abcdef
7,abcdef
8,abcdef
9,abcdef


In [7]:
# 选择每一行的首2个字符
df['text'].apply(lambda x: x[:2])

0    ab
1    ab
2    ab
3    ab
4    ab
5    ab
6    ab
7    ab
8    ab
9    ab
Name: text, dtype: object

### 在一个 dataframe 中如何在多个列上通过一个“复杂”的条件筛选行？

In [8]:
d = {'Dates':[pd.Timestamp('2013-01-02'),
              pd.Timestamp('2013-01-03'),
              pd.Timestamp('2013-01-04')],
     'Num1':[1,2,3],
     'Num2':[-1,-2,-3]}
                 

df = pd.DataFrame(data=d)
df

,Dates,Num1,Num2
0,2013-01-02,1,-1
1,2013-01-03,2,-2
2,2013-01-04,3,-3


In [9]:
# "Num1"列是正数的所有行
positive = df['Num1'] > 0

# “Num2”列上等于 -1 的行
negativeOne = df['Num2'] == -1

# 在 "Dates" 列上介于 (1/2/2013 和 1/20/2013) 之间的
Dates = df['Dates'].isin(['2013-01-02','2013-01-20'])

df[positive & negativeOne & Dates]

,Dates,Num1,Num2
0,2013-01-02,1,-1


### 如何得到一个分组里面的最大值?

In [10]:
df = pd.DataFrame({'col1':['minus','minus','positive','nan'],
                'col2':[10,20,30,40],
                'col3':[-10,-20,30,np.nan]
                })
df

,col1,col2,col3
0,minus,10,-10.0
1,minus,20,-20.0
2,positive,30,30.0
3,nan,40,NaN


In [11]:
# 方法 1
df.groupby('col1').apply(lambda x: x.max())

,col1,col2,col3
col1,,,
minus,minus,20.0,-10.0
nan,NaN,40.0,NaN
positive,positive,30.0,30.0


In [12]:
# 方法 2
df.groupby('col1').agg('max')

,col2,col3
col1,,
minus,20,-10.0
nan,40,NaN
positive,30,30.0


### 如何从一个有多层索引(multi-index)的 dataframe 中选择一层索引的记录?

In [13]:
df = pd.DataFrame({'group1' : ["a","b","a","a","b","c","c","c","c",
                            "c","a","a","a","b","b","b","b"],
                'value' : [1,2,3,4,1,3,5,6,5,4,1,2,3,4,3,2,1],
                'group2'  : ["apple","pear","orange","apple",
                            "banana","durian","lemon","lime",
                            "raspberry","durian","peach","nectarine",
                            "banana","lemon","guava","blackberry","grape"]})
df = df.set_index(['group1','group2'])
df

value
group1 group2           
a      apple           1
b      pear            2
a      orange          3
       apple           4
b      banana          1
c      durian          3
       lemon           5
       lime            6
       raspberry       5
       durian          4
a      peach           1
       nectarine       2
       banana          3
b      lemon           4
       guava           3
       blackberry      2
       grape           1

In [14]:
df.xs('a', level='group1')

,value
group2,
apple,1
orange,3
apple,4
peach,1
nectarine,2
banana,3


### 如果索引的名字和列名相同，如何重置索引?

In [15]:
df = pd.DataFrame({"Name":["Alice", "Bob", "Mallory", "Mallory", "Bob" , "Mallory"] , 
                "City":["Seattle", "Seattle", "Portland", "Seattle", "Seattle", "Portland"]}
               )
df

,City,Name
0,Seattle,Alice
1,Seattle,Bob
2,Portland,Mallory
3,Seattle,Mallory
4,Seattle,Bob
5,Portland,Mallory


In [16]:
group = df.groupby(['City','Name'])
s = group.agg('size')
s.add_suffix('_size').reset_index()

,City,Name,0
0,Portland_size,Mallory_size,2
1,Seattle_size,Alice_size,1
2,Seattle_size,Bob_size,2
3,Seattle_size,Mallory_size,1


**作者:** [David Rojas](http://www.hedaro.com/)
<p class="text-muted">本教程由<a href="http://datarx.cn" target="_blank"><strong>派兰数据</strong></a>翻译</p>